**To build a classifier using Pytorch for the given dataset**

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!unzip gdrive/My\ Drive/Dataset1.zip

Streaming output truncated to the last 5000 lines.
  inflating: Dataset 1/mountain/7539.jpg  
  inflating: Dataset 1/mountain/7551.jpg  
  inflating: Dataset 1/mountain/7560.jpg  
  inflating: Dataset 1/mountain/7565.jpg  
  inflating: Dataset 1/mountain/7578.jpg  
  inflating: Dataset 1/mountain/7581.jpg  
  inflating: Dataset 1/mountain/7586.jpg  
  inflating: Dataset 1/mountain/7647.jpg  
  inflating: Dataset 1/mountain/7652.jpg  
  inflating: Dataset 1/mountain/7654.jpg  
  inflating: Dataset 1/mountain/7662.jpg  
  inflating: Dataset 1/mountain/767.jpg  
  inflating: Dataset 1/mountain/7672.jpg  
  inflating: Dataset 1/mountain/7679.jpg  
  inflating: Dataset 1/mountain/7681.jpg  
  inflating: Dataset 1/mountain/7693.jpg  
  inflating: Dataset 1/mountain/7695.jpg  
  inflating: Dataset 1/mountain/7698.jpg  
  inflating: Dataset 1/mountain/7700.jpg  
  inflating: Dataset 1/mountain/771.jpg  
  inflating: Dataset 1/mountain/7715.jpg  
  inflating: Dataset 1/mountain/7744.jpg  
  inf

**After this we split the dataset for training, testing and predicting the type of image. And import the splitted dataset**

In [ ]:
!unzip gdrive/My\ Drive/archive.zip

Streaming output truncated to the last 5000 lines.
  inflating: seg_train/seg_train/mountain/7506.jpg  
  inflating: seg_train/seg_train/mountain/7537.jpg  
  inflating: seg_train/seg_train/mountain/7539.jpg  
  inflating: seg_train/seg_train/mountain/7551.jpg  
  inflating: seg_train/seg_train/mountain/7560.jpg  
  inflating: seg_train/seg_train/mountain/7565.jpg  
  inflating: seg_train/seg_train/mountain/7578.jpg  
  inflating: seg_train/seg_train/mountain/7581.jpg  
  inflating: seg_train/seg_train/mountain/7586.jpg  
  inflating: seg_train/seg_train/mountain/7647.jpg  
  inflating: seg_train/seg_train/mountain/7652.jpg  
  inflating: seg_train/seg_train/mountain/7654.jpg  
  inflating: seg_train/seg_train/mountain/7662.jpg  
  inflating: seg_train/seg_train/mountain/767.jpg  
  inflating: seg_train/seg_train/mountain/7672.jpg  
  inflating: seg_train/seg_train/mountain/7679.jpg  
  inflating: seg_train/seg_train/mountain/7681.jpg  
  inflating: seg_train/seg_train/mountain/7693.jp

In [ ]:
#Load libraries
import os
import numpy as np
import torch
import glob
import torch.nn as nn
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.autograd import Variable
import torchvision
import pathlib

In [ ]:
#checking for device
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
print(device)

cpu


In [ ]:
transformer=transforms.Compose([
    transforms.Resize((150,150)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),  #0-255 to 0-1, numpy to tensors
    transforms.Normalize([0.5,0.5,0.5], # 0-1 to [-1,1] , formula (x-mean)/std
                        [0.5,0.5,0.5])
])

In [ ]:
train_path='/content/seg_train/seg_train'
test_path='/content/seg_test/seg_test'

train_loader=DataLoader(
    torchvision.datasets.ImageFolder(train_path,transform=transformer),
    batch_size=64, shuffle=True
)
test_loader=DataLoader(
    torchvision.datasets.ImageFolder(test_path,transform=transformer),
    batch_size=32, shuffle=True
)

In [ ]:
#categories
root=pathlib.Path(train_path)
classes=sorted([j.name.split('/')[-1] for j in root.iterdir()])

In [ ]:
print(classes)

['buildings', 'forest', 'glacier', 'mountain', 'sea', 'street']


In [ ]:
#CNN Network


class ConvNet(nn.Module):
    def __init__(self,num_classes=6):
        super(ConvNet,self).__init__()

        #Input shape= (256,3,150,150)
        
        self.conv1=nn.Conv2d(in_channels=3,out_channels=12,kernel_size=3,stride=1,padding=1)
        #Shape= (256,12,150,150)
        self.bn1=nn.BatchNorm2d(num_features=12)
        #Shape= (256,12,150,150)
        self.relu1=nn.ReLU()
        #Shape= (256,12,150,150)
        
        self.pool=nn.MaxPool2d(kernel_size=2)
        #Reduce the image size be factor 2
        #Shape= (256,12,75,75)
        
        
        self.conv2=nn.Conv2d(in_channels=12,out_channels=20,kernel_size=3,stride=1,padding=1)
        #Shape= (256,20,75,75)
        self.relu2=nn.ReLU()
        #Shape= (256,20,75,75)
        
        
        
        self.conv3=nn.Conv2d(in_channels=20,out_channels=32,kernel_size=3,stride=1,padding=1)
        #Shape= (256,32,75,75)
        self.bn3=nn.BatchNorm2d(num_features=32)
        #Shape= (256,32,75,75)
        self.relu3=nn.ReLU()
        #Shape= (256,32,75,75)
        
        
        self.fc=nn.Linear(in_features=75 * 75 * 32,out_features=num_classes)
        
        
        
        #Feed forwad function
        
    def forward(self,input):
        output=self.conv1(input)
        output=self.bn1(output)
        output=self.relu1(output)
        output=self.pool(output)
        output=self.conv2(output)
        output=self.relu2(output)
        output=self.conv3(output)
        output=self.bn3(output)
        output=self.relu3(output)
        #Above output will be in matrix form, with shape (256,32,75,75)
            
        output=output.view(-1,32*75*75)
        output=self.fc(output)
        return output  

In [ ]:
model=ConvNet(num_classes=6).to(device)

In [ ]:
#Optmizer and loss function
optimizer=Adam(model.parameters(),lr=0.001,weight_decay=0.0001)
loss_function=nn.CrossEntropyLoss()

In [ ]:
num_epochs=20

In [ ]:
train_count=len(glob.glob(train_path+'/**/*.jpg'))
test_count=len(glob.glob(test_path+'/**/*.jpg'))
print(train_count,test_count)

14034 3000


In [ ]:
#Model training and saving best model

best_accuracy=0.0

for epoch in range(num_epochs):
    
    #Evaluation and training on training dataset
    model.train()
    train_accuracy=0.0
    train_loss=0.0
    
    for i, (images,labels) in enumerate(train_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
            
        optimizer.zero_grad()
        
        outputs=model(images)
        loss=loss_function(outputs,labels)
        loss.backward()
        optimizer.step()
        
        
        train_loss+= loss.cpu().data*images.size(0)
        _,prediction=torch.max(outputs.data,1)
        
        train_accuracy+=int(torch.sum(prediction==labels.data))
        
    train_accuracy=train_accuracy/train_count
    train_loss=train_loss/train_count
    
    
    # Evaluation on testing dataset
    model.eval()
    
    test_accuracy=0.0
    for i, (images,labels) in enumerate(test_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
            
        outputs=model(images)
        _,prediction=torch.max(outputs.data,1)
        test_accuracy+=int(torch.sum(prediction==labels.data))
    
    test_accuracy=test_accuracy/test_count
    
    
    print('Epoch: '+str(epoch)+' Train Loss: '+str(train_loss)+' Train Accuracy: '+str(train_accuracy)+' Test Accuracy: '+str(test_accuracy))
    
    #Save the best model
    if test_accuracy>best_accuracy:
        torch.save(model.state_dict(),'best_checkpoint.model')
        best_accuracy=test_accuracy
    

Epoch: 0 Train Loss: tensor(0.2365) Train Accuracy: 0.927319367250962 Test Accuracy: 0.76
Epoch: 1 Train Loss: tensor(0.1805) Train Accuracy: 0.9451332478267066 Test Accuracy: 0.748
Epoch: 2 Train Loss: tensor(0.1881) Train Accuracy: 0.9419980048453755 Test Accuracy: 0.7293333333333333
Epoch: 3 Train Loss: tensor(0.1768) Train Accuracy: 0.943708137380647 Test Accuracy: 0.7656666666666667
Epoch: 4 Train Loss: tensor(0.0970) Train Accuracy: 0.9710702579449907 Test Accuracy: 0.7023333333333334
Epoch: 5 Train Loss: tensor(0.1218) Train Accuracy: 0.9657973492945703 Test Accuracy: 0.7593333333333333
Epoch: 6 Train Loss: tensor(0.1154) Train Accuracy: 0.9653698161607525 Test Accuracy: 0.7243333333333334
Epoch: 7 Train Loss: tensor(0.1191) Train Accuracy: 0.9641584722816018 Test Accuracy: 0.7406666666666667
Epoch: 8 Train Loss: tensor(0.1080) Train Accuracy: 0.9694313809320222 Test Accuracy: 0.7173333333333334
Epoch: 9 Train Loss: tensor(0.0938) Train Accuracy: 0.9740629898817158 Test Accuracy

**We have trained the model successfully.**

In [26]:
import torch
import torch.nn as nn
from torchvision.transforms import transforms
import numpy as np
from torch.autograd import Variable
from torchvision.models import squeezenet1_1
import torch.functional as F
from io import open
import os
from PIL import Image
import pathlib
import glob
import cv2

In [27]:
train_path='/content/seg_train/seg_train'
pred_path='/content/seg_pred/seg_pred'

In [28]:
#categories
root=pathlib.Path(train_path)
classes=sorted([j.name.split('/')[-1] for j in root.iterdir()])

In [29]:
#CNN Network


class ConvNet(nn.Module):
    def __init__(self,num_classes=6):
        super(ConvNet,self).__init__()
        
        #Output size after convolution filter
        #((w-f+2P)/s) +1
        
        #Input shape= (256,3,150,150)
        
        self.conv1=nn.Conv2d(in_channels=3,out_channels=12,kernel_size=3,stride=1,padding=1)
        #Shape= (256,12,150,150)
        self.bn1=nn.BatchNorm2d(num_features=12)
        #Shape= (256,12,150,150)
        self.relu1=nn.ReLU()
        #Shape= (256,12,150,150)
        
        self.pool=nn.MaxPool2d(kernel_size=2)
        #Reduce the image size be factor 2
        #Shape= (256,12,75,75)
        
        
        self.conv2=nn.Conv2d(in_channels=12,out_channels=20,kernel_size=3,stride=1,padding=1)
        #Shape= (256,20,75,75)
        self.relu2=nn.ReLU()
        #Shape= (256,20,75,75)
        
        
        
        self.conv3=nn.Conv2d(in_channels=20,out_channels=32,kernel_size=3,stride=1,padding=1)
        #Shape= (256,32,75,75)
        self.bn3=nn.BatchNorm2d(num_features=32)
        #Shape= (256,32,75,75)
        self.relu3=nn.ReLU()
        #Shape= (256,32,75,75)
        
        
        self.fc=nn.Linear(in_features=75 * 75 * 32,out_features=num_classes)
        
        
        
        #Feed forwad function
        
    def forward(self,input):
        output=self.conv1(input)
        output=self.bn1(output)
        output=self.relu1(output)
            
        output=self.pool(output)
            
        output=self.conv2(output)
        output=self.relu2(output)
            
        output=self.conv3(output)
        output=self.bn3(output)
        output=self.relu3(output)
            
            
            #Above output will be in matrix form, with shape (256,32,75,75)
            
        output=output.view(-1,32*75*75)
            
            
        output=self.fc(output)
            
        return output

In [30]:
checkpoint=torch.load('best_checkpoint.model')
model=ConvNet(num_classes=6)
model.load_state_dict(checkpoint)
model.eval()

ConvNet(
  (conv1): Conv2d(3, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu1): ReLU()
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(12, 20, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu2): ReLU()
  (conv3): Conv2d(20, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn3): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu3): ReLU()
  (fc): Linear(in_features=180000, out_features=6, bias=True)
)

In [31]:
#Transforms
transformer=transforms.Compose([
    transforms.Resize((150,150)),
    transforms.ToTensor(),  #0-255 to 0-1, numpy to tensors
    transforms.Normalize([0.5,0.5,0.5], # 0-1 to [-1,1] , formula (x-mean)/std
                        [0.5,0.5,0.5])
])

In [32]:
#prediction function
def prediction(img_path,transformer):
    
    image=Image.open(img_path)
    
    image_tensor=transformer(image).float()
    
    
    image_tensor=image_tensor.unsqueeze_(0)
    
    if torch.cuda.is_available():
        image_tensor.cuda()
        
    input=Variable(image_tensor)
    
    
    output=model(input)
    
    index=output.data.numpy().argmax()
    
    pred=classes[index]
    
    return pred
    

In [33]:
images_path=glob.glob(pred_path+'/*.jpg')

In [34]:
pred_dict={}

for i in images_path:
    pred_dict[i[i.rfind('/')+1:]]=prediction(i,transformer)

In [35]:
pred_dict

{'101.jpg': 'glacier',
 '14919.jpg': 'buildings',
 '10366.jpg': 'sea',
 '2046.jpg': 'mountain',
 '3084.jpg': 'glacier',
 '20403.jpg': 'buildings',
 '2408.jpg': 'forest',
 '12791.jpg': 'street',
 '15219.jpg': 'buildings',
 '10676.jpg': 'sea',
 '5115.jpg': 'mountain',
 '15929.jpg': 'buildings',
 '23664.jpg': 'sea',
 '20368.jpg': 'glacier',
 '23239.jpg': 'glacier',
 '18097.jpg': 'forest',
 '19734.jpg': 'buildings',
 '5749.jpg': 'street',
 '22359.jpg': 'glacier',
 '13927.jpg': 'forest',
 '20197.jpg': 'buildings',
 '13060.jpg': 'street',
 '20005.jpg': 'glacier',
 '13845.jpg': 'forest',
 '16497.jpg': 'forest',
 '6059.jpg': 'street',
 '4674.jpg': 'sea',
 '4888.jpg': 'glacier',
 '817.jpg': 'mountain',
 '5016.jpg': 'glacier',
 '4066.jpg': 'mountain',
 '18365.jpg': 'forest',
 '23799.jpg': 'buildings',
 '560.jpg': 'forest',
 '7815.jpg': 'forest',
 '23583.jpg': 'forest',
 '17436.jpg': 'street',
 '22770.jpg': 'buildings',
 '21051.jpg': 'sea',
 '10799.jpg': 'mountain',
 '22916.jpg': 'glacier',
 '123

**Now we get the image with its type.**

**Hence we have successfully constructed a classifier for classifying the images of mountain, glacier, forest, building, sea and street.**

---

